/bin/bash: pip: command not found


In [23]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import pyjokes
from keras.models import load_model
import json
import random
import tensorflow as tf   
import pandas as pd
tf.get_logger().setLevel('ERROR')
import convert
import weather_forecast as wf    
from nltk.tag import pos_tag

In [24]:
def fetch_wikipedia(sentence):
    sentence = sentence.title()
    tagged_sent = pos_tag(sentence.split())
    pn = [word for word,pos in tagged_sent if pos == 'NNP']
    final = "".join(pn)
    return wikipedia.summary(final)

In [32]:
def get_weather(sentence):
    sentence = sentence.title()
    tagged_sent = pos_tag(sentence.split())
    pn = [word for word,pos in tagged_sent if pos == 'NNP']
    if "Weather" in pn:
        pn.remove("Weather")
    final = "".join(pn)
    print(final)
    type(wf.forecast(place = final))
print(get_weather('what is the weather of prayagraj'))

Prayagraj
{'date': '2021-01-20',
 'day': {'humidity': None,
         'narrative': None,
         'phrases': None,
         'precipitate': None,
         'temperature': None,
         'uv_description': None,
         'wind_speed': None},
 'night': {'humidity': 86,
           'narrative': 'Generally clear. Hazy. Low 11ºC. Winds W and '
                        'variable.',
           'phrases': 'Clear',
           'precipitate': 6,
           'temperature': 11,
           'uv_description': 'Low',
           'wind_speed': 8},
 'place': 'Prayagraj',
 'time': '20:47:11'}
None


In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [5]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [6]:
def predict_class(sentence, model , words , classes):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.1
    results = [[i,r] for i,r in enumerate(res) if r>=ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

In [7]:
def getResponse(intent , qtype):
    lisp = []
    if qtype == 'type1':
        data_file = open('database/intents1.json').read()
        lisp = json.loads(data_file)["type1"]
    elif qtype == 'type2':
        data_file = open('database/intents2.json').read()
        lisp = json.loads(data_file)["type2"]
    elif qtype == 'type3':
        data_file = open('database/intents3.json').read()
        lisp = json.loads(data_file)["type3"]
        
    for i in lisp:
        if(i['tag']== intent):
            result = random.choice(i['responses'])
            break
    return result

In [8]:
def call_model(sentence , qtype):
    no = qtype
    model = load_model('models/model' + no + '.h5')
    words = pickle.load(open('type' + no + '/words.pkl','rb'))
    classes = pickle.load(open('type' + no + '/classes.pkl','rb'))
    found_intent = predict_class(sentence , model , words , classes)[0]['intent']
#     print("Intent is " , found_intent)
    return found_intent

In [9]:
def main(question):
#     print("Finding the type")
    model = load_model('models/model.h5')
    words = pickle.load(open('main_model/words.pkl','rb'))
    classes = pickle.load(open('main_model/classes.pkl','rb'))
    qtype = predict_class(question , model , words , classes)[0]['intent']
#     print("\nQuestion type is" , qtype , end='\n\n')
    if qtype == 'type1':
        it = call_model(question , '1')
    elif qtype == 'type2':
        it = call_model(question , '2')
    elif qtype == 'type3':
        it = call_model(question , '3')
    return (qtype, it)

In [10]:
def solve(lisp):
    data = {}
    data['question'] = lisp
    data['intent'] = []
    data['types'] = []
    for question in lisp:
        print(question)
        a = main(question)
        data['intent'].append(a[1])
        data['types'].append(a[0])
        if a[1] == 'suggest_movie':
            print('Response : Here is a movie suggestion' , convert.give_suggestion('movies'))
        elif a[1] == 'suggest_song':
            print('Response : Here is a song suggestion' , convert.give_suggestion('songs'))
        elif a[1] == 'suggest_book':
            print('Response : Here is a book suggestion' , convert.give_suggestion('books'))
        elif a[1] == 'tell_joke':
            print("Response : Here is a joke" , pyjokes.get_joke('en'))
        elif a[1] == 'get_info':
            print("Response :" , fetch_wikipedia(question))
        else:
            typeq = a[0]
            intentq = a[1]
            print("Response :" , getResponse(intentq , typeq))
        print()
    data = pd.DataFrame(data)
    print(data)

In [12]:
lisp = ["ngl you are truly amazing"]
# lisp = ["tell me about elon musk" , "suggest me a movie" , "tell me a joke" , "hello how are you"]
solve(lisp)

ngl you are truly amazing
[{'intent': 'type1', 'probability': '1.0'}]
[{'intent': 'greeting', 'probability': '0.9644849'}]
Response : Hi there, how can I help?

                    question    intent  types
0  ngl you are truly amazing  greeting  type1
